In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-driver

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres. 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-28 14:04:23--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-03-28 14:04:23 (6.52 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Combined-Weather-Data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Load combined weather data into Spark DataFrame
from pyspark import SparkFiles
url = "https://grp2weather.s3.amazonaws.com/combined_weather.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+------------+------------+-------------+-----------+----------+
|Prov_Weather|Year_Weather|Precipitation|Temperature|Timeseries|
+------------+------------+-------------+-----------+----------+
|      Alsace|        1901|       864.18|       8.41|Historical|
|      Alsace|        1902|       737.39|       8.57|Historical|
|      Alsace|        1903|       757.38|        8.9|Historical|
|      Alsace|        1904|       704.56|       9.39|Historical|
|      Alsace|        1905|       840.87|       8.72|Historical|
|      Alsace|        1906|       729.49|       8.87|Historical|
|      Alsace|        1907|       670.79|       8.84|Historical|
|      Alsace|        1908|       791.57|       8.53|Historical|
|      Alsace|        1909|       824.61|        8.3|Historical|
|      Alsace|        1910|      1021.54|       9.06|Historical|
|      Alsace|        1911|       681.22|      10.04|Historical|
|      Alsace|        1912|       827.35|       8.97|Historical|
|      Alsace|        191

In [ ]:
# Configure settings for RDS - Group2Wineos password isn't an important password and was kept simple for this exercise.
mode = "append"
jdbc_url="jdbc:postgresql://grp2rdsinstance.cwkbfcctxu7y.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"root", 
          "password": "Group2Wineos", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write df to table in RDS
df.write.jdbc(url=jdbc_url, table='weather_data', mode=mode, properties=config)